In [1]:
# -----------------------------------------------------------Assignment Report and Code ----------------------------------------------------
# Part 1.1: Describe the dataset

#     The product category I selected is the Video Games category, with two variations of the datasets included: 5-Core reviews dataset and all reviews dataset. The 5-Core dataset contains only reviews from users and products with at least five reviews, while all reviews dataset contain every review entry, subsequently making it much more diverse. There are a total of 1324753 reviews; however, the 5-Core dataset only contains 231780 reviews. There is also a file for metadata dataset; however, that is a completely different set of information rather than a variation. Metadata contains information regarding each product. Only the 5-Core reviews dataset will be used in this analysis. This is because video games is a highly competitive market with multiple online stores available. Amazon does not have its own social platform for gaming (unlike Steam, Origin or Epic Store), making it less popular in general. However, a lot of people buy video games from Amazon when sales are happening. These customers are however unlikely to leave a review on Amazon. Nevertheless, if a user provides multiple reviews on Amazon, it is more likely that they use Amazon to buy video games in general. Therefore, I assume that their reviews should be more reliable than one-off customers. Likewise, as for video games, a lot of games are based on different platforms and people usually buy the games on their respective platforms. This means that there are much fewer copies being sold on Amazon. Products with less than five reviews are likely to have a very small share in the market and do not reflect the trend within the category.

#     The reviews and metadata can be linked together using the Amazon Standard Identification Number (ASIN) unique to each product. Both variations of the reviews dataset contains reviewerID and asin as the identifier, with other attributes being reviewerName, helpful, reviewText, overall, summary, unixReviewTime, and reviewTime. The metadata uses asin as the identifier and contains description, price, imURL, related, salesRank, categories, title, and brand. The attributes I will be using from the product data are the asin, description, price, related, salesRank, title and brand, while every attribute of the review data will be used in this analysis. Only video games product will be included in the analysis, meaning that electronics and gaming equipment will be filtered out for this report.

In [2]:
### Part 1.2: Describe the steps you used for data preparation and preprocessing
# I use the code provided at the source of the dataset to extract DataFrames from the downloaded files, then save each of them into Apache Parquet files using the df.to_parquet function in pandas. The detailed implementation of this can be found in the code written later in this notebook. 

# As for normalisation, the category data is split into multiple columns with a boolean representing each value instead of being contained in a single object. Similarly, the salesRank data will also be unwrapped from the dictionary object into a number. The brand and title of each item will also be removed as most of them do not have a value and will not contribute to the analysis. Since this analysis is focused on video games category, only the video games sales rank will be retained. Helpful column in reviews data will also be split into upvotes and downvotes. Moreover, related items will be removed from the metadata as we are not analysing the relationship between each item. UnixReviewTime will also be excluded as users are from different parts of the world; therefore they are from different timezones and it wll be hard to determine the relationship between time of the day and the reviews.


# For the metadata, items without a price is excluded as they tend to be some uncommon product people sell on the store. To clarify, standard products on Amazon should have their price listed, and the ones without are those sold by users. As a result, they are too inconsistent and unreliable for the analysis. Moreover, the metadata for the products not mentioned by the reviews in the 5-Core dataframe will also be filtered out of the analysis. ImageUrl will also be removed as the column is not required for the analysis. Reviewer Names will be excluded from the analysis as reviewerID will be used to identify each reviewer instead. Console games will be excluded as the market's growth and evolution is completely different from that of PC games.

In [3]:
### Part 1.3: Hypothesis
# Over the course of 1996 - 2014, the video games industry has greatly evolved and there has been a clear shift from physical copies of the games into digital ones; therefore, the hypothesis for this analysis is that the popularity and sample size for digital product reviews should increase over time. Additionally, sales number should also increase greatly during seasonal sales; this should be reflected in the number of reviews over specific periods.

In [4]:
# Import libraries and define parse functions for the dataset
import pandas as pd
import gzip
import sklearn

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0 
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient ='index')

In [5]:
#generate DataFrame from 5-core reviews and save to parquet
# fiveCoreDF = getDF('reviews_Video_Games_5.json.gz')
# fiveCoreDF.to_parquet('fiveCore.parquet.gzip', compression='gzip')


In [6]:
#generate DataFrame from products metadata and save to parquet
# meta = getDF('meta_Video_Games.json.gz')
# meta.to_parquet('metadata.parquet.gzip', compression='gzip')

In [7]:
# #generate DataFrame from all category reviews and save to parquet
# allDF = getDF('reviews_Video_Games.json.gz')
# allDF.to_parquet('all.parquet.gzip', compression='gzip')
# This dataset is excluded from the analysis for the reasons stated at the beginning

In [8]:
#Read metadata from parquet and store it in a variable
OG_metadata = pd.read_parquet('metadata.parquet.gzip')

In [9]:
# Read 5-Core reviews data from parquet and store it in a variable
OG_reviews= pd.read_parquet('fiveCore.parquet.gzip')

In [10]:
#Data Cleaning: Remove items without a price data from the metadata
OG_metadata = OG_metadata[OG_metadata.price.notnull()]

In [11]:
# Check asin appearing in 5-Core reviews DataFrame
reviewedGames = OG_reviews.groupby("asin").groups.keys()

# Remove items not appearing in the reviews from the metadata
metadata = OG_metadata[OG_metadata.asin.isin(reviewedGames)].reset_index()

In [12]:
#make ASIN index for metadata, remove 'index' column

metadata.set_index('asin', inplace = True)
metadata.drop(axis = 1, labels = 'index', inplace = True)

In [13]:
#remove imageurl from metadata
metadata.drop(axis = 1, labels = 'imUrl', inplace = True)
#remove title from metadata
metadata.drop(axis = 1, labels = 'title', inplace = True)
#remove brand from metadata
metadata.drop(axis = 1, labels = 'brand' , inplace = True)
#remove related items from metadata
metadata.drop(axis = 1, labels = 'related', inplace = True)

In [14]:
# metadata.replace(metadata["categories"], metadata["categories"])
def unwrapCategory(data):
    return data[0]

metadata["categories"] = metadata["categories"].apply(unwrapCategory)

In [15]:
#Unwrap metadata saleRank, retrieving only Video Games rank
temp = metadata["salesRank"].apply(pd.Series)["Video Games"]
#add unwrapped values to the original dataframe
metadata["rank"] = temp
####### Some rows do not contain Video Games Rank, it should be safe to assume that they are video games related but not actually video games. Therefore, they will be removed from the analysis

metadata = metadata.dropna(axis = 'index', how = 'any', subset = ['rank'])



In [16]:
#change metadata.rank to integer

metadata["rank"] = metadata["rank"].apply(lambda x: int(x))

#remove salesRank from the dataframe

metadata.drop(axis = 1, labels = 'salesRank', inplace = True)


In [17]:
#Delete ReviewerName from reviews
OG_reviews.drop(axis = 1, labels = 'reviewerName', inplace = True)

In [18]:
#Unwrap Helpful into Upvotes and Downvotes
helpfulDF = pd.DataFrame(OG_reviews["helpful"].to_list(), columns = ['upvotes', 'downvotes'])

#join reviews with unwrapped helpful stats, dropping helpful column
reviews = OG_reviews.join(helpfulDF)
reviews.drop(axis = 1, labels = 'helpful', inplace = True)




In [19]:
def getCategories(data):
    newItem = dict()
    for cat in data:
        if cat == 'PC':
            newItem['PC'] = True
        if cat == 'Games':
            newItem['Games'] = True
        if cat == 'Mac':
            newItem['Mac'] = True
        if cat == 'Software':
            newItem['Software'] = True
        if cat == 'Digital Game':
            newItem['Digital Game'] = True
        if cat == 'PC Game Downloads':
            newItem['PC Game Downloads'] = True
        if cat == 'Linux Games':
            newItem['Linux Games'] = True
        if cat == 'Mac Game Downloads':
            newItem['Mac Game Downloads'] = True
        if cat == 'MMO & Free-to-Play Games':
            newItem['MMO & Free-to-Play Games'] = True
    return newItem

#check unique types in metadata
test = metadata["categories"].explode()
categoriesDF = pd.DataFrame(columns=["asin", "PC", "Games", "Mac", "Software", "Digital Game", "PC Game Downloads", "Digital Games & DLC", "Linux Games", "Mac Game Downloads", "MMO & Free-to-Play Games"])



In [20]:
#unwrap selected categories, reset the index, then turn into a dataframe
test = test.groupby(["asin"]).apply(getCategories).reset_index()
test = pd.DataFrame(test)

In [21]:
#Transform categories data into a pivot table then unwrap
test = pd.pivot_table(test, values = ["categories"], index = ['asin'], columns = ['level_1'], aggfunc = lambda x: x)
test = test['categories']

In [22]:
#merge the categories data back to original metadata. 
metadata = metadata.join(test)
#delete categories column
metadata.drop(axis = 1, labels = ['categories'], inplace = True)

In [23]:
#remove non-video games items by checking if they do not belong in any of the categories (all categories value = NaN)

metadata.dropna(axis = 0, subset = ["Games", "Linux Games", "MMO & Free-to-Play Games", "Mac", "Mac Game Downloads", "PC", "PC Game Downloads", "Software"], how = "all", inplace = True)

In [24]:
#convert reviewTime to datetime, dropping unixReviewTime
reviews['reviewTime'] = pd.to_datetime(reviews['reviewTime'], format = '%m %d, %Y')
reviews.drop( axis = 1, labels = "unixReviewTime", inplace = True)

In [25]:
metadata.head(10)

,description,price,rank,Games,Linux Games,MMO & Free-to-Play Games,Mac,Mac Game Downloads,PC,PC Game Downloads,Software
asin,,,,,,,,,,,
0700099867,Dirt 3 is a popular rally racing game for Play...,246.63,6629,True,NaN,NaN,NaN,NaN,True,NaN,NaN
8176503290,Tom Clancy's H.A.W.X. 2 plunges fans into an e...,49.99,18169,True,NaN,NaN,NaN,NaN,True,NaN,NaN
907843905X,Ship Simulator 2008 puts players at the helm o...,10.00,15480,True,NaN,NaN,NaN,NaN,True,NaN,NaN
B000003SQQ,The old characters from the original tongue-in...,3.99,20150,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B000006OVE,"In most role-playing games, the player finds h...",19.99,11098,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B000006OWR,PLAY LIKE NEW..TEEN...ANIMATED VIOLENCE,24.99,16608,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B000006P0M,Used In Good Condition and Working Order,159.44,24652,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B000006P0J,This is the one that's got the press salivatin...,41.22,7902,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B000006OVI,What fighting game fan could pass up this perf...,17.98,30497,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
reviews.head(10)

,reviewerID,asin,reviewText,overall,summary,reviewTime,upvotes,downvotes
0,A2HD75EMZR8QLN,0700099867,Installing the game was a struggle (because of...,1.0,Pay to unlock content? I don't think so.,2012-07-09,8,12
1,A3UR8NLLY1ZHCX,0700099867,If you like rally cars get this game you will ...,4.0,Good rally game,2013-06-30,0,0
2,A1INA0F5CWW3J4,0700099867,1st shipment received a book instead of the ga...,1.0,Wrong key,2014-06-28,0,0
3,A1DLMTOTHQ4AST,0700099867,"I got this version instead of the PS3 version,...",3.0,"awesome game, if it did not crash frequently !!",2011-09-14,7,10
4,A361M14PU2GUEG,0700099867,I had Dirt 2 on Xbox 360 and it was an okay ga...,4.0,DIRT 3,2011-06-14,2,2
5,A2UTRVO4FDCBH6,0700099867,"Overall this is a well done racing game, with ...",4.0,"Good racing game, terrible Windows Live Requir...",2013-05-11,0,0
6,AN3YYDZAS3O1Y,0700099867,Loved playing Dirt 2 and I thought the graphic...,5.0,A step up from Dirt 2 and that is terrific!,2011-08-14,11,13
7,AQTC623NCESZW,0700099867,I can't tell you what a piece of dog**** this ...,1.0,Crash 3 is correct name AKA Microsoft,2012-11-24,1,4
8,A1QJJU33VNC4S7,0700099867,I initially gave this one star because it was ...,4.0,A great game ruined by Microsoft's account man...,2012-11-14,0,1
9,A2JLT2WY0F2HVI,0700099867,I still haven't figured this one out. Did ever...,2.0,Couldn't get this one to work,2014-02-08,1,1


In [ ]:
### Part 2: Advanced Analysis

1. Study the reviews

In [ ]:
### Part 3: Evaluation
#3.1: What are the findings of your data analytics?
#3.2: What actions for improving the products do you suggest based on your findings?
#3.3: How could you refine your data analytics?


#3.4: Are there any implications for consumers/users of the products based on the findings you obtained or the improvements you plan to make?



## The URL to the web page : https://